In [36]:
import pandas as pd
import bisect

In [3]:
from itertools import product
from math import sqrt
import numpy as np


class Heuristic():

    def __init__(self, size):
        self.size = size
        self.len = size * size

    def get_distance_one_num(self, row, col):
        pass

    def get_distance(self):
        d_map = {((row * self.size) + col) : self.get_distance_one_num(row, col)
                 for row, col in product(range(self.size), range(self.size))}
        return d_map

    def get_h(self, puzzle, solved):
        all_h = [self.dist_map[num][solved.index(puzzle[num])]
                 for num in range(self.len)]
#         print(all_h) #test
        return sum(all_h)


class Euclidian(Heuristic):

    def __init__(self, size):
        super().__init__(size)
        self.dist_map = self.get_distance()
        
    def get_distance_one_num(self, row, col):
        return [sqrt((t_row - row)**2 + (t_col - col)**2)
                    for t_row, t_col in product(range(self.size), range(self.size))]

    
class Manhattan(Heuristic):

    def __init__(self, size):
        super().__init__(size)
        self.dist_map = self.get_distance()
        
    def get_distance_one_num(self, row, col):
        return [abs(t_row - row) + abs(t_col - col)
                    for t_row, t_col in product(range(self.size), range(self.size))]

    
class Hamming(Heuristic):

    def __init__(self, size):
        super().__init__(size)
        
    def get_h(self, puzzle, solved):
        return np.count_nonzero(np.array(puzzle) - np.array(solved))

In [4]:
puzzle = [1, 2, 3, 0, 6, 4, 8, 7, 5]
target = [1, 2, 3, 8, 0, 4, 7, 6, 5]
psize = 3

In [5]:
heuristic = Manhattan(psize)

In [7]:
# type(states.iloc[0])
# current = states.iloc[0]

In [8]:
# current.name

In [6]:
def move_up(puzzle, hole, psize):
    if hole + 1 > psize:
        new = puzzle[:]
        new[hole], new[hole - psize] = new[hole - psize], new[hole]
        return new
    
def move_down(puzzle, hole, psize):
    if hole + 1 + psize <= psize * psize:
        new = puzzle[:]
        new[hole], new[hole + psize] = new[hole + psize], new[hole]
        return new
    
def move_left(puzzle, hole, psize):
    if (hole) % psize:
        new = puzzle[:]
        new[hole], new[hole - 1] = new[hole - 1], new[hole]
        return new
    
def move_right(puzzle, hole, psize):
    if (hole + 1) % psize:
        new = puzzle[:]
        new[hole], new[hole + 1] = new[hole + 1], new[hole]
        return new
    
moves = [move_up, move_down, move_left, move_right]

In [10]:
# move_left([15, 10, 14, 5, 13, 3, 0, 7, 8, 11, 1, 9, 6, 2, 12, 4],15, 4) 
move_right([15, 10, 14, 5, 13, 3, 0, 7, 8, 11, 1, 9, 6, 2, 12, 4],7, 4) 

In [73]:
# from State import State
import pandas as pd
# from moves import moves

class Solver:

    def __init__(self, puzzle, psize, target, 
                heuristic, f_calculation):
        self.initial = puzzle
        self.solved_puzzle_hash = str(target)
        self.solved_puzzle = target
        self.psize = psize
        self.states = pd.DataFrame({'puzzle': [puzzle], 'parent': None, 'g': 0,
                                    'f': heuristic.get_h(puzzle, target), 'is_open_list': 1},
                                   index=[(str(puzzle))])
        self.open_f = [self.states.f[0]]
        self.open_idx = [(str(puzzle))]
        
        # fff = self.states['f'].idxmin(axis =1)
        # fff = np.argmin(self.states[self.states.is_open_list == 1].f.values)

        # print(self.states.f.values.idxmin())
#         print(self.open_f)
#         print(self.open_idx)
        # print(heuristic.get_h(puzzle, target))
        self.heuristic = heuristic
        self.f_calculation = f_calculation
        self.comp_in_time = 0

    def a_star(self):
        # print(self.states.f.idxmin())
        while self.states['is_open_list'].sum():
#             current = self.states.iloc[np.argmin(self.states[self.states.is_open_list == 1].f.values)]
#             current = self.states.loc[self.states[self.states.is_open_list == 1].f.idxmin()]
#             current = self.states[self.states.is_open_list == 1].iloc[0]
            current = self.states.loc[self.open_idx.pop(0)]
            self.open_f.pop(0)
            if current.name == self.solved_puzzle_hash:
                return self.get_result(current)
            else:
                self.comp_in_time += 1
                self.states.loc[current.name, 'is_open_list'] = 0
                self.expand(current)
#                 self.states = self.states.sort_values('f')


    def expand(self, current):
        hole_index = current.puzzle.index(0)
        expanded = [foo(current.puzzle, hole_index, self.psize) for foo in moves]
        g = current.g + 1
        for i in expanded:
            if i:
                str_i = str(i)
                f = self.f_calculation(g, self.heuristic.get_h(i, self.solved_puzzle))
                # print(f)
                if str_i in self.states.index:
                    if f < self.states.loc[str_i]['f']:
                        self.states.loc[str_i] = [i, current.name, g, f, 1]
                        self.open_idx.insert(bisect.bisect_left(self.open_f, f), str_i)
                        bisect.insort_left(self.open_f, f)
#                         print(self.open_f)
#                         print(self.open_idx)
                else:
                    self.states.loc[str_i] = [i, current.name, g, f, 1]
                    self.open_idx.insert(bisect.bisect_left(self.open_f, f), str_i)
                    bisect.insort_left(self.open_f, f)
#                     print(self.open_f)
#                     print(self.open_idx)
                if str_i == self.solved_puzzle_hash:
                    return

    def print_states(self, idx, size):
        if self.states.loc[idx, 'parent']:
            self.print_states(self.states.loc[idx, 'parent'], size)
        print('step ', self.states.loc[idx, 'g'])
        chunks = (self.states.loc[idx, 'puzzle'][i : i + size]
               for i in range(0, size*size, size))
        for ch in chunks:
            print(ch)

    def get_result(self, current):
        print('Complexity in time: ', self.comp_in_time)
        print('Complexity in space: ', self.states.shape[0])
        print('Number of moves to solve puzzle: ', current.g)
        self.print_states(current.name, self.psize)

In [77]:
solv = Solver(puzzle, 3, target, heuristic, (lambda g, h: g + h * 1))

[4]
['[1, 2, 3, 0, 6, 4, 8, 7, 5]']


In [78]:
%time
solv.a_star()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 7.87 µs
[7]
['[0, 2, 3, 1, 6, 4, 8, 7, 5]']
[5, 7]
['[1, 2, 3, 8, 6, 4, 0, 7, 5]', '[0, 2, 3, 1, 6, 4, 8, 7, 5]']
[5, 5, 7]
['[1, 2, 3, 6, 0, 4, 8, 7, 5]', '[1, 2, 3, 8, 6, 4, 0, 7, 5]', '[0, 2, 3, 1, 6, 4, 8, 7, 5]']
[5, 7, 8]
['[1, 2, 3, 8, 6, 4, 0, 7, 5]', '[0, 2, 3, 1, 6, 4, 8, 7, 5]', '[1, 0, 3, 6, 2, 4, 8, 7, 5]']
[5, 7, 8, 8]
['[1, 2, 3, 8, 6, 4, 0, 7, 5]', '[0, 2, 3, 1, 6, 4, 8, 7, 5]', '[1, 2, 3, 6, 7, 4, 8, 0, 5]', '[1, 0, 3, 6, 2, 4, 8, 7, 5]']
[5, 7, 8, 8, 8]
['[1, 2, 3, 8, 6, 4, 0, 7, 5]', '[0, 2, 3, 1, 6, 4, 8, 7, 5]', '[1, 2, 3, 6, 4, 0, 8, 7, 5]', '[1, 2, 3, 6, 7, 4, 8, 0, 5]', '[1, 0, 3, 6, 2, 4, 8, 7, 5]']
[4, 7, 8, 8, 8]
['[1, 2, 3, 8, 6, 4, 7, 0, 5]', '[0, 2, 3, 1, 6, 4, 8, 7, 5]', '[1, 2, 3, 6, 4, 0, 8, 7, 5]', '[1, 2, 3, 6, 7, 4, 8, 0, 5]', '[1, 0, 3, 6, 2, 4, 8, 7, 5]']
[3, 7, 8, 8, 8]
['[1, 2, 3, 8, 0, 4, 7, 6, 5]', '[0, 2, 3, 1, 6, 4, 8, 7, 5]', '[1, 2, 3, 6, 4, 0, 8, 7, 5]', '[1, 2, 3, 6, 7, 4, 8, 0, 5]',

In [25]:
# solv.states.head()
# solv.states['f'].idxmin()
solv.states['f']

[1, 2, 3, 0, 6, 4, 8, 7, 5]     4
[0, 2, 3, 1, 6, 4, 8, 7, 5]     7
[1, 2, 3, 8, 6, 4, 0, 7, 5]     5
[1, 2, 3, 6, 0, 4, 8, 7, 5]     5
[2, 0, 3, 1, 6, 4, 8, 7, 5]     8
[1, 2, 3, 8, 6, 4, 7, 0, 5]     4
[1, 0, 3, 6, 2, 4, 8, 7, 5]     8
[1, 2, 3, 6, 7, 4, 8, 0, 5]     8
[1, 2, 3, 6, 4, 0, 8, 7, 5]     8
[2, 6, 3, 1, 0, 4, 8, 7, 5]     9
[2, 3, 0, 1, 6, 4, 8, 7, 5]    11
[1, 2, 3, 8, 0, 4, 7, 6, 5]     3
[0, 1, 3, 6, 2, 4, 8, 7, 5]    11
[1, 3, 0, 6, 2, 4, 8, 7, 5]    11
[1, 2, 3, 6, 7, 4, 0, 8, 5]    11
[1, 2, 3, 6, 7, 4, 8, 5, 0]    11
[1, 2, 0, 6, 4, 3, 8, 7, 5]    11
[1, 2, 3, 6, 4, 5, 8, 7, 0]    11
[2, 6, 3, 1, 7, 4, 8, 0, 5]    12
[2, 6, 3, 0, 1, 4, 8, 7, 5]    12
[2, 6, 3, 1, 4, 0, 8, 7, 5]    12
[2, 3, 4, 1, 6, 0, 8, 7, 5]    12
Name: f, dtype: int64

In [87]:
[1, 2, 3, 8, 0, 4, 7, 6, 5]
trr = move([15, 10, 5, 0, 13, 3, 14, 7, 8, 11, 1, 9, 6, 2, 12, 4], 4)     
trr
# for i in range(4):
#     print(i)

[None,
 [15, 10, 5, 7, 13, 3, 14, 0, 8, 11, 1, 9, 6, 2, 12, 4],
 [15, 10, 0, 5, 13, 3, 14, 7, 8, 11, 1, 9, 6, 2, 12, 4],
 None]

In [44]:
trr[0][0] = 'fdksl'

In [46]:
trr[0].index(0)

6

In [100]:
states.loc[str([15, 10, 0, 5, 13, 3, 14, 7, 8, 11, 1, 9, 6, 2, 12, 4])] = [[15, 10, 0, 5, 13, 3, 14, 7, 8, 11, 1, 9, 6, 2, 12, 4], ['some'], 1, 89, 1]

In [101]:
states

,puzzle,parent,g,f,is_open_list
"[15, 10, 14, 5, 13, 3, 0, 7, 8, 11, 1, 9, 6, 2, 12, 4]","[15, 10, 14, 5, 13, 3, 0, 7, 8, 11, 1, 9, 6, 2...",None,0,42,1
"[15, 10, 0, 5, 13, 3, 14, 7, 8, 11, 1, 9, 6, 2, 12, 4]","[15, 10, 0, 5, 13, 3, 14, 7, 8, 11, 1, 9, 6, 2...",[some],1,89,1


In [95]:
str([15, 10, 0, 5, 13, 3, 14, 7, 8, 11, 1, 9, 6, 2, 12, 4]) in states.index

True

In [102]:
states.loc[str([15, 10, 0, 5, 13, 3, 14, 7, 8, 11, 1, 9, 6, 2, 12, 4])]['']

89

In [37]:
l = [15, 10, 0, 5, 13, 3, 14, 7, 8, 11, 1, 9, 6, 2, 12, 4]
l.sort()
l

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [55]:
bisect.bisect_left(l, 7)
# 7

10

In [56]:
# bisect.insort_left(l, 1)
l

[0, 1, 1, 2, 3, 4, 4, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [43]:
ss = ["0", '1', '2', "3", '4', '5', '6', '7', '8']

In [44]:
ss

['0', '1', '2', '3', '4', '5', '6', '7', '8']

In [48]:
# bisect.insort_left(ss, 'fdsfa')

In [51]:
ss.insert(bisect.bisect_left(l, 7), 'adsl')
ss

['0', '1', '2', '3', '4', '4', '5', '6', 'adsl', '7', '8', 'fdsfa']

In [69]:
ss.pop(0)

'0'

In [80]:
for i in target:
    print(i, end=" ")

1 2 3 8 0 4 7 6 5 